In [82]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [83]:
hn = 3
class Model(nn.Module):
     
    def __init__(self):
        super().__init__()
        self.stepLength = 1.5
        self.hidden = torch.zeros((1, hn))
        
        self.Whh = nn.Linear(hn, hn)
        self.Whx = nn.Linear(1, hn, bias = False)
        
        self.layer1 = nn.Linear(1 + hn, 64)
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, 1)
        
        self.layer1.bias.data.fill_(0.01)
        self.layer2.bias.data.fill_(0.01)
        self.layer3.bias.data.fill_(0.01)
      
    
    def forward_RNN(self, x):
        self.hidden = torch.tanh(self.Whh(self.hidden) + self.Whx(x))
        
        
    def forward_DNN(self, x):
        
        y = torch.cat((self.hidden.repeat(x.shape[0], 1), x), 1)
        
        y = torch.relu(self.layer1(y))
        y = torch.relu(self.layer2(y))
        y = self.layer3(y)

     
        return y#*torch.exp(-0.1*x**2)
    
    def sample(self, n):
        total = 0
        x = torch.Tensor([[0]])#torch.Tensor(4*np.random.random((1,1)) - 2)
        psi_old = self.forward_DNN(x)
        
        for i in range(n):
            x_new = x + self.stepLength*torch.Tensor(2*np.random.random((1,1)) - 1)
            psi_new = self.forward_DNN(x_new)
            
            if (psi_new/psi_old)**2 > np.random.random():
                x = x_new
                psi_old = psi_new
                total += 1
            
        return x, total
    
    def resetHidden(self):
        self.hidden = torch.zeros((1, hn))
        self.forward_RNN(torch.zeros((1, 1)))
        

In [84]:
torch.manual_seed(42)
np.random.seed(42)
model = Model()
optimizer = torch.optim.Adam(model.parameters())

### Using naive minimization of energy

In [85]:
epochs = 400
N = 32
n = 20
for epoch in tqdm(range(epochs)):

    PE_acc = 0
    P_acc = 0
    E_acc = 0
    grad = 0
    
    for i in range(N):
        psi_total = 1
        
        model.resetHidden() 
        x1 = model.sample(n)[0].detach().requires_grad_()
        psi1 = model.forward_DNN(x1)
        
        model.forward_RNN(x1)
        x2 = model.sample(n)[0].detach().requires_grad_()
        psi2 = model.forward_DNN(x2)
        
        
        psi_total = psi1*psi2
        dfdx, = torch.autograd.grad(psi_total, x1, create_graph=True)
        d2fdx2, = torch.autograd.grad(dfdx, x1, create_graph=True)
        
        kinetic1 = -0.5*d2fdx2/psi_total
        potential1 = 0.5*x1**2

        
        dfdx, = torch.autograd.grad(psi_total, x2, create_graph=True)
        d2fdx2, = torch.autograd.grad(dfdx, x2, create_graph=True)
        
        kinetic2 = -0.5*d2fdx2/psi_total
        potential2 = 0.5*x2**2
        

        
        E_L = (kinetic1 + potential1 + kinetic2 + potential2 + 1/torch.abs(x1-x2)).detach()
        PE_acc += psi_total/psi_total.detach()*E_L
        P_acc  += psi_total/psi_total.detach()
        E_acc  += E_L   
    
    PE_acc /= N
    P_acc  /= N
    E_acc  /= N
    
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    
    L2 = 0
    for param in model.parameters():
        L2 += torch.sum(param**2)

    L2 /= params

    loss = 2*(PE_acc - P_acc*E_acc)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    
    
    grad = 0
    for param in model.parameters():
        grad += torch.mean(param.grad**2)
    
    
    if epoch%10 == 0:
        print(model.Whh.weight)
        print(f"epoch: {epoch}, Grad: {grad}, Energy: {E_acc.item()}, N:{N}")

  0%|          | 1/400 [00:00<01:38,  4.07it/s]

Parameter containing:
tensor([[ 0.4424,  0.4782, -0.1363],
        [ 0.5294, -0.1255,  0.1175],
        [-0.2801,  0.3381,  0.5080]], requires_grad=True)
epoch: 0, Grad: 69601.3203125, Energy: 24.34436798095703, N:32


  3%|▎         | 11/400 [00:02<01:33,  4.17it/s]

Parameter containing:
tensor([[ 0.4486,  0.4721, -0.1424],
        [ 0.5215, -0.1176,  0.1253],
        [-0.2721,  0.3301,  0.5000]], requires_grad=True)
epoch: 10, Grad: 1308.998291015625, Energy: 15.167981147766113, N:32


  5%|▌         | 21/400 [00:04<01:24,  4.47it/s]

...

 95%|█████████▌| 381/400 [01:28<00:04,  4.41it/s]

Parameter containing:
tensor([[ 0.5260,  0.3946, -0.2237],
        [ 0.4076, -0.0041,  0.2455],
        [-0.3015,  0.3596,  0.5346]], requires_grad=True)
epoch: 380, Grad: 1534.876220703125, Energy: 13.805782318115234, N:32


 98%|█████████▊| 391/400 [01:30<00:02,  4.29it/s]

Parameter containing:
tensor([[ 0.5270,  0.3936, -0.2248],
        [ 0.4067, -0.0031,  0.2466],
        [-0.3026,  0.3607,  0.5360]], requires_grad=True)
epoch: 390, Grad: 105.44309997558594, Energy: 6.506765365600586, N:32


100%|██████████| 400/400 [01:33<00:00,  4.30it/s]


In [86]:
def f(x):
    x_lin = torch.linspace(-4, 4, 100).reshape(100,-1)

    model.resetHidden()
    psi1 = model.forward_DNN(x_lin)[:,0].detach().numpy()

    x = torch.Tensor([[x]])
    #y = torch.Tensor([[y]])
    
    model.forward_RNN(x)
    psi2 = model.forward_DNN(x_lin)[:,0].detach().numpy()

    #model.forward_RNN(y)
    #psi3 = model.forward_DNN(x_lin)[:,0].detach().numpy()

    plt.plot(x_lin[:,0], psi1**2/np.sum(psi1**2), "b")
    plt.plot(x_lin[:,0], psi2**2/np.sum(psi2**2), "r")
    #plt.plot(x_lin[:,0], psi3**2/np.sum(psi3**2))
    plt.plot(x, 0.01, "bo")
    #plt.plot(y, 0.01, "ro")
    plt.xlim((-4,4))
    plt.ylim((0, 0.1))

    plt.show()
    
def g(x):
    model.resetHidden()

    x = torch.Tensor([[x]])
    #y = torch.Tensor([[y]])

    plt.plot(model.hidden.detach().numpy(), "bo")
    model.forward_RNN(x)
    #model.forward_RNN(y)

    plt.plot(model.hidden.detach().numpy(), "ro")
    plt.xlim((-7,7))
    plt.ylim((-1, 1))

    plt.show()

interact(f, x=(-4.0, 4., 0.025));
interact(g, x=(-4.0, 4., 0.1));

interactive(children=(FloatSlider(value=0.0, description='x', max=4.0, min=-4.0, step=0.025), Output()), _dom_…

interactive(children=(FloatSlider(value=0.0, description='x', max=4.0, min=-4.0), Output()), _dom_classes=('wi…

In [ ]:
model.resetHidden()
print(model.sample(20))

In [ ]:
x_lin = torch.linspace(-4, 4, 100).reshape(100,-1)
psi = model.forward(x_lin)[:,0].detach().numpy()

plt.plot(x_lin[:,0], psi**2)
plt.plot(x_lin[:,0], 1/np.sqrt(np.pi)*np.exp(-x_lin[:,0]**2), "--")
plt.legend(["DNN", "analytical"])
plt.show()


## Estimating energy

In [22]:
N = 10000

E = 0
for i in tqdm(range(N)):
    model.resetHidden() 
    x1 = model.sample(20)[0].detach().requires_grad_()
    psi1 = model.forward_DNN(x1)

    model.forward_RNN(x1)
    x2 = model.sample(20)[0].detach().requires_grad_()
    psi2 = model.forward_DNN(x2)


    psi_total = psi1*psi2
    dfdx, = torch.autograd.grad(psi_total, x1, create_graph=True)
    d2fdx2, = torch.autograd.grad(dfdx, x1, create_graph=True)

    kinetic1 = -0.5*d2fdx2/psi_total
    potential1 = 0.5*x1**2


    dfdx, = torch.autograd.grad(psi_total, x2, create_graph=True)
    d2fdx2, = torch.autograd.grad(dfdx, x2, create_graph=True)

    kinetic2 = -0.5*d2fdx2/psi_total
    potential2 = 0.5*x2**2



    E += (kinetic1 + potential1 + kinetic2 + potential2 + 1/torch.abs(x1-x2)).detach()

E = E/N

print(E.item())

100%|██████████| 10000/10000 [01:23<00:00, 120.24it/s]

1.595165491104126


In [ ]:
model.resetHidden() 
x1 = torch.Tensor([[0]]).requires_grad_()
psi1 = model.forward_DNN(x1)

model.forward_RNN(x1)
x2 = torch.Tensor([[0]]).requires_grad_()
psi2 = model.forward_DNN(x2)

psi_total = psi1*psi2
dfdx, = torch.autograd.grad(psi_total, x1, create_graph=True)
d2fdx2, = torch.autograd.grad(dfdx, x1, create_graph=True)

print(d2fdx2/psi_total)



## Testing auto grad

In [ ]:
x = [torch.Tensor([3]).requires_grad_(), torch.Tensor([6]).requires_grad_()]
y = torch.Tensor([4, 8]).requires_grad_()

f = torch.Tensor(x)**3*y**2

print(f)
#f.backward()
#print(x.grad, y.grad)

dfdx = [torch.autograd.grad(a, b, create_graph=True)[0] for a, b in zip(f, x)]
print(dfdx)

d2fdx2 = [torch.autograd.grad(a, b, create_graph=True)[0] for a, b in enumerate(dfdx, x)]

print(d2fdx2)

#d2fdx2.backward()
#print(x.grad, y.grad)

#print(dfdx)


#ddfdxdx, = torch.autograd.grad(dfdx, x, create_graph=True)
#print(ddfdxdx.grad_fn(x)[1])